In [1]:
import pandas as pd
import glob

### Import and concatenate all CO csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\CO\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
CO_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now CO_df contains all the data from the 20 CSV files 


### Data cleaning and preprocessing

In [3]:
# Drop unwanted columns
CO_df.drop(['Parameter Code', 
            'Sample Duration', 
            'Pollutant Standard', 
            'Event Type', 
            'Observation Count', 
            'Observation Percent',
            'Method Code', 
            'Method Name',  
            'Local Site Name', 
            'Address', 
            'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Aggregating so we have 1 measurement per station per day per parameter
CO_df = CO_df.groupby(['State Code', 
                       'State Name', 
                       'County Code', 
                       'County Name', 
                       'City Name', 
                       'Site Num', 
                       'Datum', 
                       'Latitude', 
                       'Longitude', 
                       'Date Local', 
                       'Parameter Name', 
                       'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first','AQI':'mean'}).reset_index()

In [5]:
# Drop Mexico, Virgin Islands and Puerto Rico
CO_df.drop(CO_df[(CO_df['State Code']==80)|(CO_df['State Code']==78)|(CO_df['State Code']==72)].index,inplace=True)

In [6]:
# Create unique identifier for each measurement
CO_df['measurement_id'] = CO_df['State Code'].apply(str) + '_' + CO_df['County Code'].apply(str) + '_' + CO_df['Site Num'].apply(str) + '_' + CO_df['Date Local'].str.replace('-','')

In [7]:
# checking for the parameters included in the dataframe
CO_df['Parameter Name'].unique()

array(['Carbon monoxide'], dtype=object)

In [8]:
# Renaming column to match the value measured and droping unnecessary columns
CO_df.rename(columns={'Arithmetic Mean':'CO(ppm)','1st Max Value':'CO(ppm) Max Value', '1st Max Hour':'CO(ppm) Max Hour'},inplace=True)
CO_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [9]:
# Convert date column to datetime
CO_df['Date Local'] = pd.to_datetime(CO_df['Date Local'])

In [10]:
CO_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,CO(ppm),CO(ppm) Max Value,CO(ppm) Max Hour,CBSA Name,AQI,measurement_id
0,1,Alabama,73,Jefferson,Birmingham,23,WGS84,33.553056,-86.815,2011-01-12,0.504084,0.80,23,"Birmingham-Hoover, AL",8.0,1_73_23_20110112
1,1,Alabama,73,Jefferson,Birmingham,23,WGS84,33.553056,-86.815,2011-01-13,0.441000,0.76,4,"Birmingham-Hoover, AL",8.0,1_73_23_20110113
2,1,Alabama,73,Jefferson,Birmingham,23,WGS84,33.553056,-86.815,2011-01-14,0.505455,1.22,23,"Birmingham-Hoover, AL",9.0,1_73_23_20110114
3,1,Alabama,73,Jefferson,Birmingham,23,WGS84,33.553056,-86.815,2011-01-15,0.817803,1.29,9,"Birmingham-Hoover, AL",13.0,1_73_23_20110115
4,1,Alabama,73,Jefferson,Birmingham,23,WGS84,33.553056,-86.815,2011-01-16,0.722992,1.08,7,"Birmingham-Hoover, AL",11.0,1_73_23_20110116


In [12]:
#Saving final df to csv
#CO_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_CO_clean.csv",index=False)
CO_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_CO_clean.csv",index=False)